ResNet model for Bacterial Colony Count Prediction

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [16]:
def residual_block(x, filters, kernel_size=3, strides=1, activation='relu'):
    y = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation(activation)(y)
    
    y = layers.Conv2D(filters, kernel_size, padding='same')(y)
    y = layers.BatchNormalization()(y)
    
    # Shortcut connection
    if strides != 1 or x.shape[-1] != filters:
        x = layers.Conv2D(filters, 1, strides=strides)(x)
    return layers.add([x, y])


In [17]:
def resnet(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Residual blocks
    num_res_blocks = [2, 2, 2, 2]  # Number of residual blocks in each stage
    filters = 64
    for i, num_blocks in enumerate(num_res_blocks):
        strides = 1 if i == 0 else 2
        x = residual_block(x, filters, strides=strides)
        for _ in range(num_blocks - 1):
            x = residual_block(x, filters)
        filters *= 2

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='linear')(x)

    model = tf.keras.Model(inputs, outputs)
    return model


In [18]:
# Defining input shape and number of classes
input_shape = (180, 240, 1)
num_classes = 1  # Colony count is a regression task

In [19]:
model = resnet(input_shape, num_classes)

In [20]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [21]:
from PIL import Image
import numpy as np

def load_and_preprocess_image(image_path, target_size=(180, 240)):

    img = Image.open(image_path)
    
    img = img.resize(target_size)
    
    # Convert image to numpy array and normalize pixel values to the range [0,1]
    img_array = np.array(img) / 255.0  
    
    return img_array

In [22]:
import numpy as np


train_image_paths = [
    "frame_00_delay-0.1s.png",
    "frame_01_delay-0.1s.png",
    "frame_02_delay-0.1s.png",
    "frame_03_delay-0.1s.png",
    "frame_04_delay-0.1s.png",
    "frame_05_delay-0.1s.png",
    "frame_06_delay-0.1s.png",
    "frame_07_delay-0.1s.png",
    "frame_08_delay-0.1s.png",
    "frame_09_delay-0.1s.png",
    "frame_10_delay-0.1s.png",
    "frame_11_delay-0.1s.png",
    "frame_12_delay-0.1s.png",
    "frame_13_delay-0.1s.png",
    "frame_14_delay-0.1s.png",
    "frame_15_delay-0.1s.png",
    "frame_16_delay-0.1s.png",
    "frame_17_delay-0.1s.png",
    "frame_18_delay-0.1s.png",
    "frame_19_delay-0.1s.png",
    "frame_20_delay-0.1s.png",
    "frame_21_delay-0.1s.png",
    "frame_22_delay-0.1s.png",
    "frame_23_delay-0.1s.png",
    "frame_24_delay-0.1s.png",
    "frame_25_delay-0.1s.png"
]
train_images = []
train_counts= [3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,6,7,7,7,8,8,8,8,8,8]

for image_path in train_image_paths:
    img = load_and_preprocess_image(image_path)  # Loading and preprocessing of image
    train_images.append(img)


train_images = np.array(train_images)

train_counts = np.array(train_counts)
train_images = train_images.reshape(-1, 180, 240, 1)



In [23]:
val_image_paths = [
    "frame_26_delay-0.1s.png",
    "frame_27_delay-0.1s.png",
    "frame_28_delay-0.1s.png",
    "frame_29_delay-0.1s.png",
    "frame_30_delay-0.1s.png"
]
val_images = []
val_counts= [8,9,9,13,13]
for image_path in val_image_paths:
    img = load_and_preprocess_image(image_path)  # Loading and preprocessing image
    val_images.append(img)
val_counts = np.array(val_counts)
val_images = np.array(val_images)
val_images = val_images.reshape(-1, 180, 240, 1)   

In [24]:
test_image_paths = [
    "frame_31_delay-0.1s.png",
    "frame_32_delay-0.1s.png",
    "frame_33_delay-0.1s.png",
    "frame_34_delay-0.1s.png",
    "frame_35_delay-0.1s.png",
    "frame_36_delay-0.1s.png",
    "frame_37_delay-0.1s.png",
    "frame_38_delay-0.1s.png",
    "frame_39_delay-0.1s.png"
]
test_images = []
test_counts= [13,15,16,16,16,16,16,17,17]
for image_path in test_image_paths:
    img = load_and_preprocess_image(image_path)  # Loading and preprocessing image
    test_images.append(img)
test_counts = np.array(test_counts)
test_images = np.array(test_images)
test_images = test_images.reshape(-1, 180, 240, 1)

In [28]:

history = model.fit(train_images, train_counts, epochs=10, batch_size=13, validation_data=(val_images, val_counts))

Epoch 1/10
2/2 [==============================] - 5s 840ms/step - loss: 7.0728 - mae: 1.8859 - val_loss: 7334.7397 - val_mae: 85.6205
Epoch 2/10
2/2 [==============================] - 1s 798ms/step - loss: 3.3763 - mae: 1.2802 - val_loss: 5601.6299 - val_mae: 74.6014
Epoch 3/10
2/2 [==============================] - 1s 804ms/step - loss: 4.3039 - mae: 1.5135 - val_loss: 10691.0996 - val_mae: 103.1902
Epoch 4/10
2/2 [==============================] - 1s 801ms/step - loss: 2.9429 - mae: 1.3383 - val_loss: 1419.5129 - val_mae: 37.2669
Epoch 5/10
2/2 [==============================] - 1s 783ms/step - loss: 1.4080 - mae: 1.0165 - val_loss: 4502.8008 - val_mae: 66.9884
Epoch 6/10
2/2 [==============================] - 1s 786ms/step - loss: 1.8795 - mae: 1.1910 - val_loss: 19108.5879 - val_mae: 138.2069
Epoch 7/10
2/2 [==============================] - 1s 783ms/step - loss: 2.1633 - mae: 1.2620 - val_loss: 28319.7227 - val_mae: 168.2718
Epoch 8/10
2/2 [==============================] - 1s 784

In [26]:
test_loss, test_mae = model.evaluate(test_images, test_counts)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')

1/1 [==============================] - 0s 171ms/step - loss: 30226.5273 - mae: 173.8319
Test Loss: 30226.52734375, Test MAE: 173.83193969726562


In [27]:

# Load image from file path
trial = Image.open("frame_38_delay-0.1s.png")

# Resize image to target size
trial = trial.resize((180, 240))

# Convert image to numpy array and normalize pixel values
trial_array = np.array(trial) / 255.0  # Normalize pixel values to range [0, 1]
trial_array = trial_array.reshape(180, 240, 1)
predicted_count = model.predict(np.expand_dims(trial_array, axis=0))[0]

print("Predicted Colony Count:", predicted_count)
  

1/1 [==============================] - 0s 424ms/step
Predicted Colony Count: [-159.68564]
